In [ ]:
####### 
# This cell is meant to install packages if some are missing
#######

# !pip install tqdm scipy
# !pip install matplotlib
# !pip install opencv-contrib-python
# !pip install numpy
# !pip install --upgrade tdmclient

In [ ]:
DEBUG = False


import matplotlib.pyplot as plt
import time
import numpy as np
from tqdm import tqdm
from kalmanfilter import KalmanFilter
import cv2   # a clean up
import local_nav as ln
import math 
from Motion_control import MotionControl
from djikstra import djikstra_algo,create_plot
from calibration import data

### Connect the thymio(the speed set to 0 is here in case of an error during calibration or during the code)
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = aw(client.wait_for_node())
aw(node.lock())


#we initialize the calibration calss aswell as some values that are needed throughout the entire code
Ts = 0.1
SPEED_L = 101
SPEED_R = 98
GND_THRESHOLD = 400

mc = MotionControl(node, client, Ts, SPEED_R, SPEED_L)

# Calibration

In this section, we will run multiple functions, in order to calibrate our thymio to the current environement. This gives us more flexibility, and makes it easier to use different thymio in different environment.

### Calibration of the thymio 

In this first step, we will use the following image to calibrate the speed, its conversion ratio from the thymio's sensor to mm/s, and its variance for the Kalman filter.

<img src="Pictures/Picture1.jpg" width="900"/>

In this next function, we will ask the thymio to move forward, at a known speed. As soon as its ground sensor detect it is on the black line, it will start a timer. When it exits the line, the timer will stop, the thymio will stop aswell, and all the desired values will be computed. In this function, the line length is know and we need to be carefully align the thymio as to make it follow the line. From this test, we can also adjust the speed on the left and right wheel to make it go as straight as possible and adjust the transition threshold if the light's intensity in the current room requires it.

The following cell needs to be executed in order to initialize the calibration class, and start the calibration process. It will then give back the values in the form of a print, and give two functions we can replace the ones mentionned earlier. We have made it this way for debugging purposes. When changing code in other files that are imported, we need to restart the jupyter kernel and rerun all the cells to initialize everything. But we already know the values of the previous calibration, and if we are in the same room, with the same map and the same thymio, we know these value shouldn't change by a large margin. The two new functions will initialize everything, paste the values found earlier, and won't run the calibration process again. And if we need to change it back, the two  new functions, for the calibration process, would again be provided.

In [ ]:
cal_data = data(Ts, SPEED_L, SPEED_R, GND_THRESHOLD, client, node, 0.329506587331065, 6.793596512574189)
cal_data.calibration_mm(mc)
mc.speed_conversion = cal_data.speed_conversion

### Calibration of the camera
In this next cell, we will retrieve the the grid from the camera, and do all the conversions requiered for the rest of the project. 

We initialize the vision class here, with a calibration function, because of the light's density in our current room. We often needed to adjust the threshold for multiple detection process in this class. We could've automated the process, but have decided against it, as it never was a time-consuming task. We would usually run the cell bellow with the DEBUG defined as true, check the number of obstacles, and from this, we immediatly knew what to do with our threshold value, and change it once, or twice at the very maximum.

In [ ]:
# be careful that if you have not called the previous cell, this will automatically set the missing values
nbAruco = 2
threshold = 100
calibrate = False
vision, Q_cam, R_cam= cal_data.cam_calibration(calibrate, nbAruco, threshold)

HALF_CELL_WIDTH = vision.cell_width/2
if(DEBUG == True):
    print("Number of obstacles = ", np.count_nonzero(vision.grid))
    print(vision.grid)
    lines = vision.show()
    plt.figure
    plt.title('Lines')
    plt.imshow(cv2.cvtColor(lines.astype('uint8'), cv2.COLOR_BGR2RGB))


### Kalman's filter

The filter we have chosen to use is the kalman's filter. With the use of the camera, and the thymio's speed sensor, we have two different sensor with which we can use the Kalman's filter. It's implementation is traditionnal. Again, we have gone for the usage of a class, as it makes it easier to use, and track the values. 
The one section where we had room to play with, were the values of the covariance of process noise's matrices Q, and the values of the covariance of the measurement's noise R.

 $R = \begin{bmatrix} r_p & 0\\ 0 & r_\nu \end{bmatrix}$
 $Q = \begin{bmatrix} q_p & 0\\ 0 & q_\nu \end{bmatrix}$

The values have been computed using the same method as in the exercise of week 8. The $r_\nu$ and $q_\nu$ have been computed inside the calibration process. For $q_p$ and $r_p$, it was a little bit more difficult. An issue we had, was the deformation of the image due to the lens, and the position of the camera. 

<img src="Pictures/Camera_pos.jpg" />

When placing the camera, we always placed it perfectly in the middle of the grid along its x axis, but because of our setup, we could never place it in the middle of its width along the y axis aswell. It was always on the side. This induced an error on the y position of the thymio. We have fixed it by placing a y_offset in the vision class. With this offset, we managed to correct the position in y from the camera's perspective, but it wasn't perfect either. We still had an error of approximatively ±5mm. Based on the exercise of week 8 again, we decided to set $r_p$ = 0.01 But while testing this, we also managed to see the variance on the position from the camera's perspective. We found from this that the standard deviation was ~0.5mm, so we have set $q_p$ =  0.25mm


For this filter, we've inspired ourselves from the courses and the youtuber "L42" and his [github](https://github.com/L42Project/Tutoriels/tree/master/Divers/tutoriel36), to properly understand the filter. 


### Run code

Now that everything has been explained, we simply need to run the cell bellow to execute thymio's task.



In [ ]:
vision.update_coordinates()
if(DEBUG == True):
    print("thymio real pos : ", vision.thymio_real_pos)
    print("thymio pos in grid : ", vision.thymio_position)
    print("goal pos : ", vision.goal_position)
    print("thymio angle = :", vision.thymio_orientation)
vision.grid[vision.thymio_position[1]][vision.thymio_position[0]] = 0
vision.grid[vision.goal_position[1]][vision.goal_position[0]] = 0
shortest_path = djikstra_algo(vision.grid.T, vision.thymio_position, vision.goal_position)
if(DEBUG == True):
    print(shortest_path)

KF = KalmanFilter(Ts, vision.thymio_real_pos, cal_data.speed_conversion, Q_cam, R_cam)  # we initialize the filter

In [ ]:

mc.step_duration = HALF_CELL_WIDTH*2 / (mc.SPEED_AVG * mc.speed_conversion)
mc.turn_duration = 98 / (mc.SPEED_AVG * mc.speed_conversion)
restart = True

jump_x, jump_y = 0,0
MARGIN = 40
change_dir = False
while (restart == True):
    jump = False
    index = 0
    restart = False
    vision.update_coordinates()
    if(DEBUG == True):
        print("thymio real pos : ", vision.thymio_real_pos)
        print("thymio pos in grid : ", vision.thymio_position)
        print("goal pos : ", vision.goal_position)
        print("thymio angle = :", vision.thymio_orientation)
    vision.grid[vision.thymio_position[1]][vision.thymio_position[0]] = 0
    vision.grid[vision.goal_position[1]][vision.goal_position[0]] = 0
    shortest_path = djikstra_algo(vision.grid.T, vision.thymio_position, vision.goal_position)
    if(DEBUG == True):
        print(shortest_path)
    KF = KalmanFilter(Ts, vision.thymio_real_pos, cal_data.speed_conversion, Q_cam, R_cam)
    mc.orientation = mc.correct_orientation(vision.thymio_orientation)
    x = vision.thymio_position[0]
    y = vision.thymio_position[1]
    speed = np.array([SPEED_L, SPEED_R])
    turn_speed = np.array([0, 0])
    for dx,dy in np.transpose(shortest_path):
        if jump:
            x = dx      #actualize the coordinates of the robot
            y = dy      #actualize the coordinates of the robot
            index += 1
            if jump_x == dx and jump_y == dy:
                jump = False
            continue  
        vision.update_coordinates()
        if(vision.goal_position != vision.goal_previous) & (change_dir != True):
            change_dir = True
            restart = True
            break
        turn = mc.get_turn(dx-x,dy-y,mc.orientation)
        mc.orientation = (mc.orientation + turn)%4
        for i in range(abs(turn)):
            mc.robot_turn(np.sign(turn))
        mc.adjust_angle(vision)
        if (((dx-x)!=0) | ((dy-y)!=0)):
            local = ln.obstacle_detect(node)
            if local:
                if(DEBUG == True):
                    print("obstacle",len(shortest_path[1]))
                jump,jump_x,jump_y = ln.obstacle_avoid(vision, mc, x, y, shortest_path, index, node, client)
            else:   
                if (((dx-x)!=0) | ((dy-y)!=0)):
                    step_done = False
                    start_move = time.time()
                    mc.motors(speed[0], speed[1])
                    temp = 0
                    next_target_x = dx *HALF_CELL_WIDTH*2 + HALF_CELL_WIDTH
                    next_target_y = (vision.rows - 1 - dy) *HALF_CELL_WIDTH*2 + HALF_CELL_WIDTH
                    
                    while (step_done != True):  
                        vision.update_coordinates()
                        kalman_pos= KF.filter(vision.thymio, vision.thymio_real_pos, speed, vision.thymio_orientation)
                        if(DEBUG == True):
                            print("estimated position ", kalman_pos)
                            print("position from camera ", vision.thymio_real_pos)
                        delta_x, delta_y= mc.kalman_adjust(next_target_x, next_target_y, kalman_pos, vision.thymio_orientation)
                        current = time.time()
                        temp = current - start_move
                        if((np.abs(delta_x) < MARGIN) & (np.abs(delta_y) < MARGIN)):
                            step_done = True
                            mc.motors(0, 0) 
                        elif(temp > mc.step_duration):
                            step_done = True  
                            mc.motors(0, 0)
                
                    mc.adjust_angle(vision)

        x = dx      #actualize the coordinates of the robot
        y = dy      #actualize the coordinates of the robot
        index += 1

### Kalman_adjust

With the kalman filter, we get an estimated position of our thymio, from our multiple sensor. But we still need to use these informations. Because of our environement and path finding choices, we have decided to run the Kalman_adjust per cell. This means that we will run the correction *per steps*. We take the center of our thymio from the kalman's estimated position, and take the difference with the center of the next targeted cell. The goal is for the thymio to face the center of this cell. This is what Kalman_adjust do. It computes the angle required to reach the center of the next targeted cell, compares it with the actual thymio's orientation, and runs a proportionnal controller to fix the error. We found two approaches: in both case, we increase the speed of one wheel, while we decrease the speed of the other. This will lead to a rotation of the thymio. Then we could either decide to let this different in speed be, and expect kalman to correct the speed at every sampling time, or, the second approach, the one we went with, set this difference in speed, and let it run for a little time, then reset the speed back to normal. We have decided to set the duration to two sampling time, to be sure it would be long enough for it to affect the locomotion. The first option has also been tested, but resulted in a lot of oscillation from the thymio.

The gain was set experimentally. We know that the $k_p * \omega * t$  ($\omega$ being the angular velocity) shouldn't exceed the difference in angle from the desired orientation and the actual one. We could've increased the gain up to this limit but found the value of 2.5 was good enough, without being too agressive.
Finally, we have also made the choice of ignoring a difference in angle bellow our threshold of 7°, because of this imperfection of our setup, and the camera's deformation.

### Conclusion

This project being made of random groups, and over a very short time, forced us to cooperate and communicate properly. We managed to work all together, and help each other face problem we all faced. The task was certainly not always easy, but having this cooperation inside the group was a key aspect of the work we have done.

We've also learned, with a practical point of view, how path finding, local navigation, image processing and the filtering actually works

We have set a defined goal for this project and feel we have managed to succeed. Surely, we could've made some aspect of our project more raffined, ellaborate, and/or versatile, but given the time available for this project, shared with other project with other group, we are happy with the results.
